In [11]:
# 使用TFLearn来实现预测的正弦函数
# 标准的循环神经网络模型预测的是离散的数值，所以需要在程序中将连续的sin函数曲线离散化
import numpy as np
import tensorflow as tf

import matplotlib as mpl
mpl.use("Agg")
from matplotlib import pyplot as plt

learn = tf.contrib.learn

HIDDEN_SIZE = 30    # LSTM中隐藏节点的个数
NUM_LAYERS = 2      # LSTM的层数
BATCH_SIZE = 32     # batch大小
TIME_STEPS = 10     # 循环神经网络的截断长度
TRAINING_STEPS = 10000    # 训练轮数

TRAINING_EXAMPLES = 10000    # 训练数据个数
TESTING_EXAMPLES = 1000      # 测试数据个数
SAMPLE_GAP = 0.01            # 采样间隔

def generate_data(seq):
    X = []
    Y = []
    #  序列的第i项和后面的TIME_STEPS-1项合在一起作为输入；第i+TIME_STEPS项作为输出。
    # 即用sin函数的前TIME_STEPS个点的信息，预测第i+TIME_STEPS个点的函数值
    for i in range(len(seq) - TIME_STEPS - 1):
        X.append(seq[i: i + TIME_STEPS])
        Y.append(seq[i + TIME_STEPS])
        
    return np.array(X, dtype=np.float32), np.array(Y, dtype=np.float32)
 

# 自定义多层lstm模型
def lstm_model(X, Y):
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(HIDDEN_SIZE)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * NUM_LAYERS)
    # 按列重新生成集合, unpack在0.10版本以上支持
#     x_ = tf.unpack(X, axis=1)
    
    # 使用TensorFlow接口将多层的LSTM结构连接成RNN网络并计算前向传播结果
    output, _ = tf.nn.rnn(cell, X, dtype=tf.float32)
    
    # 在本问题中只关心最后一个时刻的输出结果，该结果为下一时刻的预测值
    output = output[-1]
    
    # 对LSTM网络的输出再加一层全连接层并计算损失。注意这里默认的损失为平均平方差损失函数
    prediction, loss = learn.models.linear_regression(output, y)
    
    # 创建模型优化器并得到优化步骤
    train_op = tf.contrib.layers.optimize_loss(
        loss,
        tf.contrib.framework.get_global_step(),
        optimizer="Adagrad",
        learn_rate=0.01
    )
    return prediction, loss, train_op

# 建立深层循环网络模型
regressor = learn.Estimator(model_fn=lstm_model)

# 用正弦函数生成训练和测试数据集合
# numpy.linspace函数可以创建一个等差数列的数组，常用的有三个参数：第一个表示起始值，第二个表示终止值，死俺哥表示数列长度
test_start = TRAINING_EXAMPLES * SAMPLE_GAP
test_end = (TRAINING_EXAMPLES + TESTING_EXAMPLES) * SAMPLE_GAP
train_X, train_Y = generate_data(np.linspace(0, test_start, TRAINING_EXAMPLES, dtype=np.float32))
test_X, test_Y = generate_data(np.linspace(test_start, test_end, TESTING_EXAMPLES, dtype=np.float32))

# 通过fit函数训练模型
regressor.fit(train_X, train_Y, batch_size=BATCH_SIZE, steps=TRAINING_STEPS)

#使用训练好的模型对测试数据进行预测
predicted = [[pred] for pred in regressor.predict(test_X)]

# 计算rmse作为评价指标
rmse = np.sqrt(((predicted - test_Y) ** 2).mean(axis=0))
print "Mean Square Error is: %f" % rmse[0]

# 对预测的函数曲线进行绘图，并存储到运行目录下的sin.png
fig = plt.figure()
plot_predicted = plt.plot(predicted, label='predicted')
plot_test = plt.plot(test_Y, label='real_sin')
plot.legend([plot_predicted, plot_test], ['predicted', 'real_sin'])
# plot.show()
# fig.savefig('sin.png')
    
    

TypeError: inputs must be a list